<h1 id="ALG_004_EXP_002-Baseline-MacLean_et_al-param_search">Analysis on the parameter "match"'s effect on disparity accuracy</h1>
<h2 id="Experiment-notebook:-ALG_004_EXP_002-Baseline-MacLean_et_al-param_search">Experiment notebook: <a href="ALG_004_EXP_002-Baseline-MacLean_et_al-param_search.ipynb" target="_blank" rel="noopener">ALG_004_EXP_002-Baseline-MacLean_et_al-param_search</a></h2>
<h2 id="Introduction">Introduction</h2>
<p>The assumption that the value of the match parameter contributes to streaking artefacts was hypothesized in&nbsp; <a href="ALG_001-EXP_001-Baseline-VIS.ipynb" target="_blank" rel="noopener">ALG_001-EXP_001-Baseline</a>. While it had been stated that the lack of inter-scanline constraints were responsible for such artefacts (Scharstein, Szeliski and Zabih, 2001), this idea was intended to be investigated further in this experiment.</p>
<h2 id="Abstract">Abstract</h2>
<p>The assumption that simply by changing one parameter&rsquo;s, match&rsquo;s value would reduce streaking artefacts (Hirschmuller, 2008) were tested further on two scenes of the Middlebury 2003 dataset. While the results due to the size of the dataset were not statistically substantiated, clear correlation between the match value and streaking artefacts (and as a result, measured error) was visually observed. Comparing different metrics was not conclusive as they had contradictory results. Further research was decided into this matter.</p>
<h2 id="Relevant-theory">Relevant theory</h2>
<p>The streaking artefacts ( see below for further details) are regarded as&nbsp; the symptomatic effect of scanline-wise dynamic programming approaches (Scharstein, Szeliski and Zabih, 2001; Daniel Scharstein and Heiko Hirschm&uuml;ller, 2014; Madeo <em>et al.</em>, 2016). This effect is thought to be the result of such algorithm&rsquo;s lack of considering inter-scanline constraints. For instance, Hirschm&uuml;ller reported significant reduction of such artefacts by introducing additional non-horizontal comparison &nbsp;lines when computing cost. Madeo et al. achieved reduction of such artefacts by applying a median filter post-processing stage to the pipeline. While the author of this report did not intend to argue the effectiveness of the aforementioned methods, it was deemed to be worth&nbsp; further research whether such artefacts could be reduced by changing the parameters.</p>
<h2 id="Method">Method</h2>
<p>Similarly, to previous experiments, two scenes of the Middlebury 2003 dataset were used. Match, gap and egap values were initialised to 60, -20, -1. Then in steps of 1, the value of match was reduced&nbsp; output disparity was benchmarked. The disparity images were saved, the measured accuracy was logged. Then, plotting the results on respective graphs by scenes, the results were evaluated using an interactive visualisation tool that displayed the corresponding disparity to the plotted data point ( see below for further details).</p>
<h2 id="Results-and-discussion">Results and discussion</h2>
<p>Upon visual analysis, the streaking artefacts were the least observable when using match values between 25 and 30 for both scenes. However, when comparing the plots of different metrics, the results were not conclusive. For instance, &ldquo;abs_error&rdquo; which is the sum of the absolute differences between the output disparity and ground truth, local minimum was observable within the aforementioned range. However, when evaluating bad metric, with the threshold of 2 (0.5 pixel accuracy) for instance, this local minimum is not reached when it comes to the scene &ldquo;Teddy&rdquo;. If Eucledian distance or Mean Squared Error (mse) &nbsp;was used the effect was the opposite, &ldquo;Teddy&rdquo; had a local minimum while &ldquo;Cones&rdquo; did not have. This was deemed as highly inconclusive ( see below for further details).</p>
<h2 id="Conclusion">Conclusion</h2>
<p>The effects of the parameter match were tested and the assumption that the streaking artefacts could be reduced by changing the parameter&rsquo;s value was supported by the results. However, the results of different metrics were inconclusive. Further analysis on a larger dataset would be needed to establish statistically well-founded findings.</p>
<h2 id="References">References</h2>
<ol>
<li>Daniel Scharstein and Heiko Hirschm&uuml;ller (2014) <em>Submit | Middlebury Stereo Evaluation - Version 3</em>. Available at: http://vision.middlebury.edu/stereo/submit3/ (Accessed: 17 October 2019).</li>
<li>Hirschmuller, H. (2008) &lsquo;Stereo Processing by Semiglobal Matching and Mutual Information&rsquo;, <em>IEEE Transactions on Pattern Analysis and Machine Intelligence</em>, 30(2), pp. 328&ndash;341. doi: 10.1109/TPAMI.2007.1166.</li>
<li>Madeo, S. <em>et al.</em> (2016) &lsquo;An optimized stereo vision implementation for embedded systems: application to RGB and infra-red images&rsquo;, <em>Journal of Real-Time Image Processing</em>, 12(4), pp. 725&ndash;746. doi: 10.1007/s11554-014-0461-7.</li>
<li>Scharstein, D., Szeliski, R. and Zabih, R. (2001) &lsquo;A taxonomy and evaluation of dense two-frame stereo correspondence algorithms&rsquo;, in <em>Proceedings IEEE Workshop on Stereo and Multi-Baseline Vision (SMBV 2001)</em>. IEEE Comput. Soc, pp. 131&ndash;140. doi: 10.1109/SMBV.2001.988771.</li>
</ol>

In [1]:
import sys
import os
sys.path.append(os.path.join("..", ".."))
import pandas as pd
import ipywidgets as widgets
import numpy as np
import glob
import cv2
import plotly.graph_objs as go
import plotly.express as px
from ipywidgets import HBox, VBox, Button, Image, Layout

from components.utils import plotly_helpers as plh

In [2]:
log_directory = os.path.join("..", "logs")
csv_files = glob.glob(os.path.join(log_directory, "ALG_004_*.csv"))
filenames = [os.path.split(csv_file)[-1] for csv_file in csv_files]
opts = [(f,p) for f, p in zip(filenames, csv_files) if "well_known_filters" not in f]

csv_selector = widgets.Dropdown(
    options=opts,
    description='CSV Log file:',
    value=os.path.join('..','logs', 'ALG_004_EXP_002-Baseline-MacLean_et_al-Numba_param_search.csv')
)

available_metrix = ['abs_error',
       'mse', 'avg', 'eucledian', 'bad1', 'bad2', 'bad4', 'bad8']
metrics_selector = widgets.Dropdown(
    options=[(m,m) for m in available_metrix],
    description='Default metric:'
)

In [3]:
VBox([csv_selector, metrics_selector])

In [4]:
selected_file = os.path.abspath(csv_selector.value)

df = plh.load_n_clean(selected_file, gts=False, kernel_sizes=False)
df = df.sort_values(by=metrics_selector.value)

teddy = df_temp_1 = df[df["scene"]=="teddy"]
cones = df_temp_2 =  df[df["scene"]!="teddy"]

img_widget_teddy = Image(value=df["loaded_imgs"].iloc[0], 
                   layout=Layout(height='375px', width='450px'))
img_widget_cones = Image(value=df["loaded_imgs"].iloc[0], 
                   layout=Layout(height='375px', width='450px'))


In [5]:
# Getting Figure widgets and dataframes
fig_teddy, dfs_1 = plh.get_figure_widget_traced(teddy, "match",
                                                metrics_selector.value, "are_occlusions_errors", discrete_hover=False)

fig_teddy.layout.title="scene Teddy"
fig_cones, dfs_2 = plh.get_figure_widget_traced(cones, "match", 
                                                metrics_selector.value, "are_occlusions_errors", discrete_hover=True)

fig_cones.layout.title="scene Cones"
plh.bind_hover_function_for_traced([fig_teddy], img_widget_teddy, dfs_1)
plh.bind_hover_function_for_traced([fig_cones], img_widget_cones, dfs_2)

dashboard = VBox([HBox([fig_teddy, img_widget_teddy]), 
                 HBox([fig_cones, img_widget_cones])]
                  )


In [6]:
display(dashboard)

    'data': [{'hovertemplate': '%{text}',
              'name': '…

In [7]:
VBox([csv_selector, metrics_selector])

#### "Cones" best results

In [8]:
df_temp_1.sort_values(by=metrics_selector.value, inplace=True)
df_temp_2.sort_values(by=metrics_selector.value, inplace=True)
df_temp_1[["match", "gap", "egap", "scene", "kernel_size", metrics_selector.value]].head(10)

<ipython-input-8-35733564e03b>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-8-35733564e03b>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,match,gap,egap,scene,kernel_size,abs_error
142,35,-20,-1,teddy,1,1032737
134,33,-20,-1,teddy,1,1034417
130,32,-20,-1,teddy,1,1035785
138,34,-20,-1,teddy,1,1037099
126,31,-20,-1,teddy,1,1039331
122,30,-20,-1,teddy,1,1043737
118,29,-20,-1,teddy,1,1047235
114,28,-20,-1,teddy,1,1051995
98,24,-20,-1,teddy,1,1053267
102,25,-20,-1,teddy,1,1054931


#### "Teddy" best results

In [9]:
df_temp_2[["match", "gap", "egap", "scene", "kernel_size", metrics_selector.value]].head(10)

,match,gap,egap,scene,kernel_size,abs_error
116,28,-20,-1,cones,1,1450700
120,29,-20,-1,cones,1,1450996
100,24,-20,-1,cones,1,1451266
124,30,-20,-1,cones,1,1452730
128,31,-20,-1,cones,1,1452740
132,32,-20,-1,cones,1,1455244
112,27,-20,-1,cones,1,1456162
104,25,-20,-1,cones,1,1456966
92,22,-20,-1,cones,1,1459082
96,23,-20,-1,cones,1,1460124
